# 1.Pré tratamento dos dados para análise.

In [365]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

In [366]:
df=pd.read_csv(r"Cuspidão\Completao.csv",delimiter=';',encoding="latin-1",decimal=',')

# Criar variáveis gerais

In [367]:
#Cria um variável com o nome de todos os setores do dataset e cria uma flag para se tem somente essa empresa no setor
ListofCompanies=df.drop_duplicates(subset=['SETOR_ATIV','Nome Empresa'])
ListofCompanies=ListofCompanies['SETOR_ATIV'].value_counts()
ListofCompanies=pd.DataFrame(ListofCompanies)
Listof1Companies=ListofCompanies[ListofCompanies['SETOR_ATIV']==1]
ListofCompanies=ListofCompanies[ListofCompanies['SETOR_ATIV']>2]
ListofCompanies['Flag1Company']=0
Listof1Companies['Flag1Company']=1
ListofCompanies=pd.concat([ListofCompanies,Listof1Companies])
ListofCompanies['SETOR_ATIV']=ListofCompanies.index

#Cria Variáveis de Cuspir
DF_Polyline=pd.DataFrame()
DF_Cuspidao=pd.DataFrame()
DF_Cuspidao2=pd.DataFrame()
DF_Cuspidao_BAU=pd.DataFrame()
DF_Cuspidao_RESTO=pd.DataFrame()
df_produ_kmeans_REST_ALL=pd.DataFrame()

# Pré tratar dados para modelos


In [368]:
#Criar ano e mes
df['Ano']=df['DT_REFER'].apply(str).str[0:4]
df['Mes']=df['DT_REFER'].apply(str).str[5:7]
#Transformar colunas negativas em positivas
df['Passivo Circulante']=df['Passivo Circulante']*-1
df['Passivo Total']=df['Passivo Total']*-1
#Coloca a Flag de 1 company no dataset central
df1 = pd.merge( ListofCompanies,df, left_on='SETOR_ATIV', right_on='SETOR_ATIV')


In [369]:
df_produ=df1

In [370]:
#Transforma as relações de indicadores para clusterizar em numeros, para o for andar de lado.

df_produ[14]=df_produ['Ativo Circulante'].clip(lower=0)
df_produ[13]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[12]=df_produ['Passivo Circulante'].clip(upper=0)
df_produ[11]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[10]=df_produ['Passivo Total'].clip(upper=0)
df_produ[9]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[8]=df_produ['Ativo Total'].clip(lower=0)
df_produ[7]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[6]=df_produ['Reservas de Lucro'].clip(lower=0)
df_produ[5]=df_produ['Receita'].clip(lower=0)
df_produ[4]=df_produ['Receita'].clip(lower=0)
df_produ[3]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[2]=df_produ['Receita'].clip(lower=0)
df_produ[1]=df_produ['Lucro/Prejuízo do Período'].clip(lower=0)




# Clusterizar as empresas com dados realizados

In [371]:
df_provisorio=df_produ
df_provisorio=df_provisorio[['Nome Empresa',
                             'Ticker',
                             'SETOR_ATIV',
                             'Flag1Company',
                             'DT_REFER',
                             'Ano',
                             'Mes',
                             1,2,3,4,5,6,7,8,9,10,11,12,13,14]]
#BAU KMEANS
for y in range(1,13):
    for x in ListofCompanies.index.values:
        if y==1:
            y1=1
            y2=2
        else:
            y1=y+1
            y2=y+2
        #filtra o setor
        df_produ_kmeans=df_provisorio[(df_provisorio['SETOR_ATIV']==x)&(df_provisorio['Flag1Company']==0)].reset_index(drop=True)
        AnoMAX=df_produ_kmeans['DT_REFER'].apply(str).str[0:4].max()
        MesMAX=df_produ_kmeans['DT_REFER'].apply(str).str[5:7].max()
        #Filtrar ano e mês (ultimo tri de cada setor)
        df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans["Ano"]==AnoMAX)&(df_produ_kmeans["Mes"]==MesMAX)].reset_index(drop=True)      
        #Clusteriza as empresas com valores negativos como 0 (ruim)
        df_produ_kmeans_REST_NO=df_produ_kmeans[(df_produ_kmeans[y1]==0)|(df_produ_kmeans[y2]==0)].reset_index(drop=True)
        df_produ_kmeans_REST_NO[f"{y1}{y2} Cluster"]=0
        #Separa as empresas que são elegíveis a escalabilidade e a clusterização
        df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans[y1]!=0)&(df_produ_kmeans[y2]!=0)].reset_index(drop=True)
        df_produ_kmeans=df_produ_kmeans.sort_values(by=[y1,y2]).reset_index(drop=True)
        #se tiver mais de 1 amostra (empresa)
        if len(df_produ_kmeans)>=2:
            #FAÇO A ESCALABILIDADE COM STANDARD
            scaler = StandardScaler() 
            pca = PCA(n_components = 2) 
            features= df_produ_kmeans[[y1,y2]]
            scaled_df = scaler.fit_transform(features) 
            normalized_df = normalize(scaled_df) 
            normalized_df = pd.DataFrame(normalized_df)
            #E DEPOIS REDUZO AS DIMENSÕES E CORRELAÇÕES COM O PCA
            X_principal = pca.fit_transform(normalized_df) 
            features = pd.DataFrame(X_principal) 
            features.columns = ['P1', 'P2']
            c=3
            if len(features)<3:
                c=len(features)
            kmeans = KMeans(n_clusters=c, init='random',
                        n_init=10,
                        max_iter=300,random_state=0)
            kmeans.fit(features)
            d = KMeans(n_clusters=c, init='random',
                        n_init=10,
                        max_iter=300,random_state=0).fit_predict(features)
            df_produ_kmeans[f"{y1}{y2} Cluster"]=d
            DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
            DF_Cuspidao_BAU=pd.concat([DF_Cuspidao2,DF_Cuspidao_BAU]).reset_index(drop=True)
        else:
            DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
            DF_Cuspidao_BAU=pd.concat([DF_Cuspidao2,DF_Cuspidao_BAU]).reset_index(drop=True)
    df_provisorio=DF_Cuspidao_BAU
    DF_Cuspidao2=pd.DataFrame()
    DF_Cuspidao_BAU=pd.DataFrame()
    
DF_Cuspidao_BAU=df_provisorio
DF_Cuspidao=pd.DataFrame()
df_provisorio=df_produ
df_provisorio=df_provisorio[['Nome Empresa',
                             'Ticker',
                             'SETOR_ATIV',
                             'Flag1Company',
                             'DT_REFER',
                             'Ano',
                             'Mes',
                             1,2,3,4,5,6,7,8,9,10,11,12,13,14]]
#REJEITADOS KMEANS
for y in range(1,13):
    if y==1:
        y1=1
        y2=2
    else:
        y1=y+1
        y2=y+2
    #filtra o setor
    df_produ_kmeans=df_provisorio[(df_provisorio['Flag1Company']==1)].reset_index(drop=True)
    AnoMAX=df_produ_kmeans['DT_REFER'].apply(str).str[0:4].max()
    MesMAX=df_produ_kmeans['DT_REFER'].apply(str).str[5:7].max()
    #Filtrar ano e mês (ultimo tri de cada setor)
    df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans["Ano"]==AnoMAX)&(df_produ_kmeans["Mes"]==MesMAX)].reset_index(drop=True)
    #Clusteriza as empresas com valores negativos como 0 (ruim)
    df_produ_kmeans_REST_NO=df_produ_kmeans[(df_produ_kmeans[y1]==0)|(df_produ_kmeans[y2]==0)].reset_index(drop=True)
    df_produ_kmeans_REST_NO[f"{y1}{y2} Cluster"]=0
    #Separa as empresas que são elegíveis a escalabilidade e a clusterização
    df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans[y1]!=0)&(df_produ_kmeans[y2]!=0)].reset_index(drop=True)
    df_produ_kmeans=df_produ_kmeans.sort_values(by=[y1,y2]).reset_index(drop=True)
    #se tiver mais de 1 amostra (empresa)
    if len(df_produ_kmeans)>=2:
        #FAÇO A ESCALABILIDADE COM STANDARD
        scaler = StandardScaler() 
        pca = PCA(n_components = 2) 
        features= df_produ_kmeans[[y1,y2]]
        scaled_df = scaler.fit_transform(features) 
        normalized_df = normalize(scaled_df) 
        normalized_df = pd.DataFrame(normalized_df)
        #E DEPOIS REDUZO AS DIMENSÕES E CORRELAÇÕES COM O PCA
        X_principal = pca.fit_transform(normalized_df) 
        features = pd.DataFrame(X_principal) 
        features.columns = ['P1', 'P2']
        c=3
        if len(features)<3:
            c=len(features)
        kmeans=0
        kmeans = KMeans(n_clusters=c, init='random',
                    n_init=10,
                    max_iter=300,random_state=0)
        kmeans.fit(features)
        d = KMeans(n_clusters=c, init='random',
                    n_init=10,
                    max_iter=300,random_state=0).fit_predict(features)
        df_produ_kmeans[f"{y1}{y2} Cluster"]=d
        DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
        DF_Cuspidao_RESTO=pd.concat([DF_Cuspidao2,DF_Cuspidao_RESTO]).reset_index(drop=True)
    else:
        DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
        DF_Cuspidao_RESTO=pd.concat([DF_Cuspidao2,DF_Cuspidao_RESTO]).reset_index(drop=True)
    df_provisorio=DF_Cuspidao_RESTO
    DF_Cuspidao2=pd.DataFrame()
    DF_Cuspidao_RESTO=pd.DataFrame()
DF_kmeans=pd.concat([DF_Cuspidao_BAU,df_provisorio]).reset_index(drop=True)
DF_kmeans["Dados"]="Realizado"

In [372]:
DF_kmeans=DF_kmeans[['Nome Empresa','Ticker','SETOR_ATIV','DT_REFER',"Dados",
             1,2,3,4,5,6,7,8,9,10,11,12,13,14,
            '12 Cluster',
            '34 Cluster',
            '56 Cluster',
            '78 Cluster',
            '910 Cluster',
            '1112 Cluster',
            '1314 Cluster',
             ]]


# Prever os próximos 4 tri

In [373]:
df_produ=df1

In [374]:
df_produ[14]=df_produ['Ativo Circulante']
df_produ[13]=df_produ['Patrimônio Líquido']
df_produ[12]=df_produ['Passivo Circulante']
df_produ[11]=df_produ['Patrimônio Líquido']
df_produ[10]=df_produ['Passivo Total']
df_produ[9]=df_produ['Patrimônio Líquido']
df_produ[8]=df_produ['Ativo Total']
df_produ[7]=df_produ['Patrimônio Líquido']
df_produ[6]=df_produ['Reservas de Lucro']
df_produ[5]=df_produ['Receita']
df_produ[4]=df_produ['Receita']
df_produ[3]=df_produ['Patrimônio Líquido']
df_produ[2]=df_produ['Receita']
df_produ[1]=df_produ['Lucro/Prejuízo do Período']


In [375]:
df_provisorio=df_produ
df_provisorio=df_provisorio[['Nome Empresa',
                             'Ticker',
                             'SETOR_ATIV',
                             'Flag1Company',
                             'DT_REFER',
                             'Ano',
                             'Mes',
                             1,2,3,4,5,6,7,8,9,10,11,12,13,14]]
LitsofTickers=df_provisorio.drop_duplicates(subset='Ticker')
LitsofTickers=LitsofTickers['Ticker'].values
for x in LitsofTickers:
    DF_Poly=df_provisorio[df_provisorio['Ticker']==x].reset_index(drop=True)
    DF_Poly.sort_values(by=['DT_REFER']).reset_index(drop=True)
    DF_Poly["indexes"]=DF_Poly.index
    DF_Poly['DT_REFER']=pd.to_datetime(DF_Poly['DT_REFER'])
    Lastmonth=DF_Poly.iloc[-1]['DT_REFER']
    Lastname=DF_Poly.iloc[-1]['Nome Empresa']
    Lastsector=DF_Poly.iloc[-1]['SETOR_ATIV']
    Lastnamesub=DF_Poly.iloc[-1]['Ticker']
    Lastindex=DF_Poly.iloc[-1]['indexes']
    Lastmonth1=Lastmonth+ relativedelta(months=+3)
    Lastmonth2=Lastmonth+ relativedelta(months=+6)
    Lastmonth3=Lastmonth+ relativedelta(months=+9)
    Lastmonth4=Lastmonth+ relativedelta(months=+12)
    Dateto=pd.DataFrame({'Data_REF': [Lastmonth1,Lastmonth2,Lastmonth3,Lastmonth4,]})
    X_seq=pd.DataFrame({'x': [Lastindex+1,Lastindex+2,Lastindex+3,Lastindex+4,]})
    polyreg=make_pipeline(PolynomialFeatures(3),LinearRegression())
    Pred=Dateto
    Pred["Nome Empresa"]=Lastname
    Pred['Ticker']=Lastnamesub
    Pred['SETOR_ATIV']=Lastsector
    for y in range(1,15):
        yy=DF_Poly[['indexes']]
        xx=DF_Poly[[y]]
        polyreg.fit(yy,xx)
        Pred2=polyreg.predict(X_seq)
        Pred2=pd.DataFrame(Pred2,columns=[y])
        Pred[y]=Pred2
    DF_Polyline=pd.concat([Pred,DF_Polyline]).reset_index(drop=True)

    
DF_Polyline['Ano']=DF_Polyline['Data_REF'].apply(str).str[0:4]
DF_Polyline['Mes']=DF_Polyline['Data_REF'].apply(str).str[5:7]

In [ ]:
#DF_Cuspidao=DF_Cuspidao.fillna(0)
#DF_Cuspidao.to_csv(rf"Cuspidão\K_Cluster_companies.csv",decimal=',',sep=';', encoding='latin-1',index=False)